In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
val_data = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
sample = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
data

In [ ]:
val_data

In [ ]:
val_data.tail()

In [ ]:
sample

In [ ]:
val_data.less_toxic

In [ ]:
less_toxic_comments = val_data.less_toxic
d_less = {
    'comments':less_toxic_comments,
}
less_toxic= pd.DataFrame(d_less)
less_toxic["toxic"] = "less_toxic"
less_toxic

In [ ]:
more_toxic = val_data.more_toxic
d_more = {
    'comments':more_toxic,
}

more_toxic= pd.DataFrame(d_more)
more_toxic["toxic"] = "more_toxic"
more_toxic

In [ ]:
train = pd.concat([less_toxic,more_toxic])

In [ ]:
train.toxic.value_counts()

In [ ]:
train_d = train.sample(frac=1, random_state=42).reset_index()
train_d

In [ ]:
train_d['toxic'] = train_d['toxic'].apply(lambda x: 1 if x=='more_toxic' else 0)
train_d

In [ ]:
train_d = train_d[['comments','toxic']]
cleaned_data = train_d.drop_duplicates()
cleaned_data.tail()

In [ ]:
train_d.tail()

by unclean data



1.   by uncleaned max data (simple tensorflow) -> [0.6857806444168091, 0.641813337802887]
2.   by cleand data (simple tensorflow) -> 



In [ ]:
from sklearn.model_selection import train_test_split

train_data_m,val_data_m,train_labels_m,val_labels_m = train_test_split(train_d['comments'].to_numpy(),
                                                               train_d['toxic'].to_numpy(),
                                                               test_size=0.2,
                                                               random_state=41)

train_data_m.shape,val_data_m.shape,train_labels_m.shape,val_labels_m.shape

In [ ]:
from sklearn.model_selection import train_test_split

train_data,val_data,train_labels,val_labels = train_test_split(cleaned_data['comments'].to_numpy(),
                                                               cleaned_data['toxic'].to_numpy(),
                                                               test_size=0.2,
                                                               random_state=41)

train_data.shape,val_data.shape,train_labels.shape,val_labels.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Setup text vectorization with custom variables
max_vocab_length = 50000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
train_vac = text_vectorizer.adapt(train_d.comments)
train_vac

In [ ]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim=max_vocab_length,
                      output_dim=128,
                       embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [ ]:
input = layers.Input(shape=(1,),dtype='string',name='input_layer')
vectorized = text_vectorizer(input)
embedded = embedding(vectorized)

x = layers.GlobalAveragePooling1D()(embedded)

output = layers.Dense(1,activation='sigmoid',name="output_layer")(x)

model_1 = keras.Model(input,output,name='model_1')


In [ ]:
model_1.compile(loss="binary_crossentropy",
              optimizer= tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history_model_1_c = model_1.fit(train_data_m,train_labels_m,
                              # validation_data=(val_data,val_labels),
                              epochs=5)

In [ ]:
model_1.evaluate(val_data,val_labels)

In [ ]:
val_pred_ = model_1.predict(val_data)
val_pred_ = tf.squeeze(tf.round(val_pred_))
val_pred_

In [ ]:
predicted = model_1.predict(data.text)
predicted

In [ ]:
d = np.array(tf.round(predicted))

In [ ]:
data1 = data.drop(columns=['text'])

In [ ]:
data1['score'] = d
data1.head()

By comparing both model we can say that tensorflow model_1 predicted well.

In [ ]:
data1.to_csv("submission.csv",index = False)